# Animate GPX tracks on the map

The goal is to help aligning a raster orienteering map with GPS, align GPX tracks temporally and display them as a head-to-head race.

In [ ]:
from time import sleep
from itertools import chain, tee
import base64
import io
from ipyleaflet import *
from IPython.display import display, FileLink
import ipywidgets as widgets
import gpxpy
from PIL import Image
import numpy as np
import cv2
import folium
from folium.plugins import TimestampedGeoJson


class Tracks:
    def __init__(self, files):
        self.points = [self.get_data(f) for f in files]
        self.center = None
        self.topleft = None
        self.botright = None
        self.calc_extents()

    def get_data(self, file):
        points = []
        gpx = gpxpy.parse(file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append((point.latitude, point.longitude,
                                   point.speed, point.time))
        return points

    def calc_extents(self):
        t = tee(chain.from_iterable(self.points), 4)
        topleft = (min(p[0] for p in t[0]), min(p[1] for p in t[1]))
        botright = (max(p[0] for p in t[2]), max(p[1] for p in t[3]))
        width = botright[0] - topleft[0]
        height = botright[1] - topleft[1]
        self.center = (topleft[0] + 0.5 * width, topleft[1] + 0.5 * height)
        margin = (width * 0.1, height * 0.1)
        self.topleft = (topleft[0] - margin[0], topleft[1] - margin[1])
        self.botright = (botright[0] + margin[0], botright[1] + margin[1])

class ImgMap:
    def __init__(self, img, tracks):
        self.img = img
        self.tracks = tracks
        self.image_file = Image.open(io.BytesIO(self.img))

    def get_data(self):
        data = 'data:image/png;base64,'
        data += base64.b64encode(self.img).decode('utf-8')
        return data

    def get_size(self):
        return self.image_file.size

    def get_xy(self, lat, lon):
        w, h = self.get_size()
        tracks = self.tracks
        x = w * (lon - tracks.topleft[1]) / (tracks.botright[1] - tracks.topleft[1])
        y = h * (tracks.botright[0] - lat) / (tracks.botright[0] - tracks.topleft[0])
        return x, y
    
    def get_for_cv(self):
        return cv2.cvtColor(np.array(self.image_file), cv2.COLOR_RGB2BGR)

class Anchors:
    def __init__(self, img_map, tracks):
        self.m = Map(center=tracks.center, zoom=13)

        layers_control = LayersControl(position='topright')
        self.m.add_control(layers_control)

        # Image layer
        image = ImageOverlay(
            url=img_map.get_data(),
            bounds=(tracks.topleft, tracks.botright),
        )
        lg1 = LayerGroup(name='Image markers', layers=(image, ))

        icon1 = AwesomeIcon(
            name='thumbtack',
            marker_color='red',
            icon_color='black',
            spin=False
        )

        self.image_markers = []
        for lat in (tracks.topleft[0], tracks.botright[0]):
            for long in (tracks.topleft[1], tracks.botright[1]):
                mark = Marker(location=(lat, long), icon=icon1)
                lg1.add_layer(mark)
                self.image_markers.append(mark)
        self.m.add_layer(lg1)

        opacity_slider = widgets.FloatSlider(description='Image:', min=0.0, max=1.0, value=1.0)
        widgets.jslink((opacity_slider, 'value'), (image, 'opacity'))
        opacity_control = WidgetControl(widget=opacity_slider, position='topright')
        self.m.add_control(opacity_control)

        # Track layer
        track = Polyline(
            locations=[[p[0], p[1]] for p in tracks.points[0]],
            color='red',
            weight=2,
            fill=False,
            opacity=0.6
        )
        lg2 = LayerGroup(name='Map markers', layers=(track, ))

        icon2 = AwesomeIcon(
            name='bullseye',
            marker_color='blue',
            icon_color='black',
            spin=False
        )

        self.map_markers = []
        for lat in (tracks.topleft[0], tracks.botright[0]):
            for long in (tracks.topleft[1], tracks.botright[1]):
                mark = Marker(location=(lat, long), icon=icon2)
                lg2.add_layer(mark)
                self.map_markers.append(mark)
        self.m.add_layer(lg2)


def GetAlignedImage(anchors, img_map):
    src_points = np.float32([img_map.get_xy(*p.location) for p in anchors.image_markers])
    dst_points = np.float32([img_map.get_xy(*p.location) for p in anchors.map_markers])
    mat = cv2.getPerspectiveTransform(src_points, dst_points)
    img = img_map.get_for_cv()
    img = cv2.warpPerspective(img, mat, img_map.get_size())
    is_ok, aligned_map = cv2.imencode('.jpg', img)
    if not is_ok:
        raise "Failed to align the image"
    return aligned_map


class FinalMap:
    def __init__(self, tracks, aligned_map):
        self.m = folium.Map(location=tracks.center, zoom_start=14)

        if not aligned_map is None:
            data = 'data:image/jpeg;base64,'
            data += base64.b64encode(aligned_map).decode('utf-8')
            img = folium.raster_layers.ImageOverlay(
                name="Orienteering map",
                image=data,
                bounds=[tracks.topleft, tracks.botright],
                opacity=1,
                interactive=False,
                cross_origin=False,
                zindex=1,
            )
            img.add_to(self.m)

        colors = ['red', 'blue', 'green', 'brown', 'darkcyan']

        data = {
            "type": "FeatureCollection",
            "features": []
        }

        def add_track(color, points, dt):
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString',
                    'coordinates': [[p[1], p[0]] for p in points]
                },
                'properties': {
                    'times': [(p[3].timestamp() + dt) * 1000 for p in points],
                    'style': {
                        'color': color,
                        'opacity': 0.5,
                    },
                }
            }
            data['features'].append(feature)


        for i, points in enumerate(tracks.points):
            dt = (tracks.points[0][0][3] - points[0][3]).total_seconds()
            add_track(colors[i], points, dt)

        TimestampedGeoJson(data, period='PT5S', transition_time=50).add_to(self.m)
        folium.LayerControl().add_to(self.m)

In [ ]:
class App:
    def __init__(self):
        self.tracks = None
        self.aligned_map = None
        self.img_map = None
        self.anchors = None
        self.final_map = None
        self.anchors_out = widgets.Output() #layout={'border': '1px solid black'})
        self.map_out = widgets.Output() #layout={'border': '1px solid black'})

    def get_download_btn(self):
        download_btn = widgets.Button(description="Get the link")
        self.file_link = None
        def on_download_btn_clicked(b):
            if self.file_link:
                return
            self.final_map.m.save('map.html')
            with self.map_out:
                self.file_link = FileLink('map.html')
                display(self.file_link)
        download_btn.on_click(on_download_btn_clicked)
        return download_btn
        
    def set_tracks(self, files):
        self.map_out.clear_output()
        self.tracks = Tracks(files)
        self.final_map = FinalMap(self.tracks, self.aligned_map)

        download_btn = self.get_download_btn()
        with self.map_out:
            display(self.final_map.m, download_btn)

    def set_image(self, img):
        self.img_map = ImgMap(img, self.tracks)
        self.anchors = Anchors(self.img_map, self.tracks)
        
        plot_btn = widgets.Button(description="Plot the map")
        def on_plot_btn_clicked(b):
            self.map_out.clear_output()
            self.aligned_map = GetAlignedImage(self.anchors, self.img_map)
            self.final_map = FinalMap(self.tracks, self.aligned_map)
            download_btn = self.get_download_btn()
            with self.map_out:
                display(app.final_map.m, download_btn)
        plot_btn.on_click(on_plot_btn_clicked)

        self.anchors_out.clear_output()
        with self.anchors_out:
            display(self.anchors.m, plot_btn)

app = App()

In [ ]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

## 1. Select a couple of GPX tracks

The working are will be determined from the first track.

In [ ]:
gpx_upload = widgets.FileUpload(accept='.gpx', multiple=True)
display(gpx_upload)

#@debounce(0.2)
def on_tracks_value_change(change):
    app.set_tracks([file['content'] for file in gpx_upload.value.values()])

gpx_upload.observe(on_tracks_value_change, names='value')

## 2. Now upload a raster image of the orienteering map


Select features on the image as far from each other as possible.
Then mark the same features on the map.

If this step is omitted, there will be no overlay.

In [ ]:
img_upload = widgets.FileUpload(accept='image/*', multiple=False)
display(img_upload, app.anchors_out)

#@debounce(0.2)
def on_img_value_change(change):
    img = next(iter(img_upload.value.values()), None)
    if img:
        app.set_image(img['content'])

img_upload.observe(on_img_value_change, names='value')

## 3. Get the map with the aligned image map and track race

In [ ]:
display(app.map_out)